# EDA
> EDA란 수집된 데이터에 대해 통계 및 대표값 추출 등을 통해 다양한 각도에서 관찰하고 이해할 수 있도록 하는 분석 기법을 의미합니다.

## 목차
- import Base code
    - representative_value / 대푯값
    - quantile / 사분위수
    - outlier / 이상치
    - scatterplot / 산점도
    - corr / 상관관계
- 함수 시연
    - representative_value / 대푯값
    - quantile / 사분위수
    - outlier / 이상치
    - scatterplot / 산점도
    - corr / 상관관계

## import Base code


In [33]:
import pandas as pd
import numpy as np


def type_mapping(field_type):
    support = {
        "int64": "INTEGER",
        "float64": "REAL",
        "datetime64": "TIMESTAMP",
        "object": "TEXT",
        np.dtype('float64'): "REAL",
        np.dtype('int32'): "INTEGER",
        np.dtype('int64'): "INTEGER",
        np.dtype('object'): "TEXT"
    }
    python_type = support[field_type]
    return python_type

def parse_df(df):
    if isinstance(df, pd.DataFrame):
        fields = [{"name": a[0], "type": type_mapping(a[1])} for a in zip(df.columns.tolist(), df.dtypes.apply(lambda x: x.name).to_list())]
        return {
            "fields": fields,
            "results": df.values.tolist()
        }
    else:
        return df

## representative_value / 대푯값
> 객체의 기초 통계 정보를 요약해주는 함수입니다. 평균, 표준 편차, 최대/최소값, 사분위수 등을 계산하여 반환합니다.

- 구성
  -  representative_value 실행함수

In [34]:
# representative_value 실행함수

def representative_value(df: pd.DataFrame):
    df = df.describe()
    df = df.reset_index(level=0)
    df.rename(columns={
        'index': '기준열'
    }, inplace=True)
    return df

## quantile / 사분위수
> 4분위수(quartile)란, 데이터를 4개의 집단으로 나누는 것을 의미합니다. 4분위수는 아래와 같은 4가지로 나뉩니다.
  첫번째 사분위수 (Q1) : 데이터의 25%
  두번째 사분위수 (Q2) : 데이터의 50% (이를 중앙값이라고 합니다)
  세번째 사분위수 (Q3) : 데이터의 75%
  네번째 사분위수 (Q4) : 데이터의 100%

- 구성
    - quantile 실행함수

In [35]:
# quantile 실행함수

def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f

    return decorator


def q_at(target, y):
    @rename(f'{target}_Q{y}')
    def q(x):
        return x.quantile(float(f"0.{y}"))

    return q


def quantile(df: pd.DataFrame):
    by, target, rate = "기준", "값(숫자)", 25
    quantile_list = []
    for percent in range(0, 101, int(rate)):
        quantile_list.append(q_at(target, percent))
    df = df.groupby(by).agg(quantile_list).reset_index(col_level=1)
    df.columns = df.columns.droplevel(level=0)
    df.rename(columns={
        '기준': '기준열', '값(숫자)_Q0': "최솟값", '값(숫자)_Q25': "1사분위",
        '값(숫자)_Q50': '중앙값', '값(숫자)_Q75': '3사분위', '값(숫자)_Q100': '최댓값'
    }, inplace=True)
    return df

## outlier / 이상치
> iqr(interquartile range) 이상치는 통계학에서 사용되는 개념입니다. 이는 데이터의 사분위수(quartile)를 기반으로 하여 이상치를 검출하는 방법입니다.

- 구성
    - outlier 실행함수

In [36]:
# outlier 실행함수

def outlier(df: pd.DataFrame):
    group_by, target = "기준", "값(숫자)"
    target_type = type_mapping(df["값(숫자)"].dtype)

    q3 = df[target].quantile(0.75)
    q1 = df[target].quantile(0.25)
    iqr = q3 - q1

    def iqr_outlier(df):
        if df[target] > q3 + 1.5 * iqr or df[target] < q1 - 1.5 * iqr:
            return df[target]
        else:
            return None

    df['이상치'] = df.apply(iqr_outlier, axis=1)
    df.rename(columns={
        '기준': '범주', '값(숫자)': "값"
    }, inplace=True)
    df = df.replace({np.nan: None})
    res = []
    for i in df.values.tolist():
        tmp = []
        for j in i:
            if j is pd.NA:
                tmp.append(None)
            else:
                tmp.append(j)
        res.append(tmp)
    return {
        "fields": [
            {"name": "범주", "type": "TEXT"},
            {"name": "값", "type": target_type},
            {"name": "이상치", "type": target_type}],
        "results": res,
        "message": None
    }

## scatterplot / 산점도
> DataFrame 객체에서 x, y 열을 지정하여 산점도(Scatter Plot)를 그릴 수 있는 기능입니다.

- 구성
    -  scatterplot 실행함수

In [37]:
# scatterplot 실행함수

def scatterplot(df: pd.DataFrame):
    df.rename(columns={
        '값(숫자)': '값'
    }, inplace=True)
    return df

## corr / 상관관계
>  상관관계 분석은 Pearson 상관계수를 계산합니다.

- 구성
    - corr 실행함수

In [38]:
# corr 실행함수

def corr(df: pd.DataFrame):
    df = df.corr(method='pearson')
    df = df.reset_index(level=0)
    df.rename(columns={
        'index': 'X'
    }, inplace=True)
    df = df.replace({np.nan: None})

    res = []
    for i in df.values.tolist():
        tmp = []
        for j in i:
            if isinstance(j, float):
                tmp.append(round(j, 3))
            else:
                tmp.append(j)
        res.append(tmp)
    return {
        "fields": [{"name": a[0], "type": type_mapping(a[1])}
                   for a in zip(df.columns.tolist(), df.dtypes.apply(lambda x: x.name).to_list())],
        "results": res,
        "message": None
    }

# 함수 시연
## representative_value / 대푯값

In [39]:
column = [
    "값1(숫자)", "값2(숫자)", "값3(숫자)", "값4(숫자)", "값5(숫자)"
]
value = [
[5.1,	3.5,	1.4,	0.2,	0.2],
[4.9,	3.0,	1.4,	0.2,	0.2],
[4.7,	3.2,	1.3,	0.2,	0.2],
[4.6,	3.1,	1.5,	0.2,	0.2]
]

test_df = pd.DataFrame(value, columns=column)
result = representative_value(test_df)
print(parse_df(result))

{'fields': [{'name': '기준열', 'type': 'TEXT'}, {'name': '값1(숫자)', 'type': 'REAL'}, {'name': '값2(숫자)', 'type': 'REAL'}, {'name': '값3(숫자)', 'type': 'REAL'}, {'name': '값4(숫자)', 'type': 'REAL'}, {'name': '값5(숫자)', 'type': 'REAL'}], 'results': [['count', 4.0, 4.0, 4.0, 4.0, 4.0], ['mean', 4.824999999999999, 3.1999999999999997, 1.4, 0.2, 0.2], ['std', 0.22173557826083448, 0.21602468994692867, 0.08164965809277258, 0.0, 0.0], ['min', 4.6, 3.0, 1.3, 0.2, 0.2], ['25%', 4.675, 3.075, 1.375, 0.2, 0.2], ['50%', 4.800000000000001, 3.1500000000000004, 1.4, 0.2, 0.2], ['75%', 4.95, 3.2750000000000004, 1.4249999999999998, 0.2, 0.2], ['max', 5.1, 3.5, 1.5, 0.2, 0.2]]}


## quantile / 사분위수

In [40]:
column = [
    "기준", "값(숫자)"
]
value = [
["일", 1598],
["월", 1978],
["화", 2032],
["수", 2255],
["목", 2103],
["금", 2556],
["토", 2387],
]

test_df = pd.DataFrame(value, columns=column)
result = quantile(test_df)
print(parse_df(result))

{'fields': [{'name': '기준열', 'type': 'TEXT'}, {'name': '최솟값', 'type': 'REAL'}, {'name': '1사분위', 'type': 'REAL'}, {'name': '중앙값', 'type': 'REAL'}, {'name': '3사분위', 'type': 'REAL'}, {'name': '최댓값', 'type': 'REAL'}], 'results': [['금', 2556.0, 2556.0, 2556.0, 2556.0, 2556.0], ['목', 2103.0, 2103.0, 2103.0, 2103.0, 2103.0], ['수', 2255.0, 2255.0, 2255.0, 2255.0, 2255.0], ['월', 1978.0, 1978.0, 1978.0, 1978.0, 1978.0], ['일', 1598.0, 1598.0, 1598.0, 1598.0, 1598.0], ['토', 2387.0, 2387.0, 2387.0, 2387.0, 2387.0], ['화', 2032.0, 2032.0, 2032.0, 2032.0, 2032.0]]}


## outlier / 이상치

In [41]:
column = [
    "기준", "값(숫자)"
]
value = [
["일", 1598],
["월", 12],
["화", 2032],
["수", 2255],
["목", 2103],
["금", 4000],
["토", 2387],
]

test_df = pd.DataFrame(value, columns=column)
result = outlier(test_df)
print(result)


{'fields': [{'name': '범주', 'type': 'TEXT'}, {'name': '값', 'type': 'INTEGER'}, {'name': '이상치', 'type': 'INTEGER'}], 'results': [['일', 1598, None], ['월', 12, 12.0], ['화', 2032, None], ['수', 2255, None], ['목', 2103, None], ['금', 4000, 4000.0], ['토', 2387, None]], 'message': None}


## scatterplot / 산점도

In [42]:
column = [
    "범주", "값(숫자)"
]
value = [
[1807,	114],
[2035,	162],
[2097,	156],
[2342,	185],
[2199,	161],
[2804,	177],
[2857,	179],
[1768,	140]
]

test_df = pd.DataFrame(value, columns=column)
result = scatterplot(test_df)
print(parse_df(result))

{'fields': [{'name': '범주', 'type': 'INTEGER'}, {'name': '값', 'type': 'INTEGER'}], 'results': [[1807, 114], [2035, 162], [2097, 156], [2342, 185], [2199, 161], [2804, 177], [2857, 179], [1768, 140]]}


## scatterplot / 산점도

In [43]:
column = [
    "값1(숫자)", "값2(숫자)", "값3(숫자)", "값4(숫자)", "값5(숫자)"
]
value = [
[5.1,	3.5,	1.4,	0.2,	0.2],
[4.9,	3.0,	1.4,	0.2,	0.2],
[4.7,	3.2,	1.3,	0.2,	0.2],
[4.6,	3.1,	1.5,	0.2,	0.2]
]

test_df = pd.DataFrame(value, columns=column)
result = corr(test_df)
print(result)

{'fields': [{'name': 'X', 'type': 'TEXT'}, {'name': '값1(숫자)', 'type': 'TEXT'}, {'name': '값2(숫자)', 'type': 'TEXT'}, {'name': '값3(숫자)', 'type': 'TEXT'}, {'name': '값4(숫자)', 'type': 'TEXT'}, {'name': '값5(숫자)', 'type': 'TEXT'}], 'results': [['값1(숫자)', 1.0, 0.626, -0.184, None, None], ['값2(숫자)', 0.626, 1.0, -0.189, None, None], ['값3(숫자)', -0.184, -0.189, 1.0, None, None], ['값4(숫자)', None, None, None, None, None], ['값5(숫자)', None, None, None, None, None]], 'message': None}
